In [4]:
import google.generativeai as genai
import pandas as pd
import os
import time
import csv
import io
import requests # <--- MAKE SURE THIS LINE IS HERE
from google.api_core import exceptions
import re
from dotenv import load_dotenv

# Carrega as variáveis do arquivo .env
load_dotenv()

# Tenta carregar a chave de API
API_KEY = os.getenv('API_KEY')

if API_KEY:
    print("Chave de API carregada com sucesso!")
    # Use a variável 'api_key' aqui
else:
    print("Erro: A chave de API não foi encontrada.")
# ... rest of the script
"""

"""
# --- Configuration ---

OUTPUT_CSV_FILE = 'only_not modern_slavery.csv' # Optional: Filename to save Block 2 CSV
BLOCK1_OUTPUT_DIR = ''
# --- Column names in your input CSV ---
# Adjust these if your column names are
id_column = 'id'
text_column = 'content'
name_column = 'name'

# --- Gemini Model Configuration ---
MODEL_NAME = 'gemini-1.5-flash' # Or 'gemini-pro' or other suitable model
genai.configure(api_key=API_KEY)

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"}, # Be cautious adjusting these
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

model = genai.GenerativeModel(MODEL_NAME, safety_settings=safety_settings) # Apply safety settings
generation_config = genai.GenerationConfig(temperature=0.2)



INPUT_CSV_URL  = "https://media.githubusercontent.com/media/anasofia0/lii_of_india_scraper/refs/heads/main/lii_india_modern_slavery.csv"


SUPPLY_CHAIN_DEF = "\"Supply chain refers to the linked set of resources and processes between and among multiple levels of an enterprise, each of which is an acquirer that begins with the sourcing of products and services and extends through the product and service life cycle\" NIST 800.161"
MODERN_SLAVERY_DEF = "\"Recently, the term “modern slavery” has increasingly been used as an umbrella term to describe various forms of coercion, including slavery, forced labor, and human trafficking\""
MODERN_SLAVERY_IN_SUPPLY_CHAIN = "IF there is a victim of forced labor, child labor, human trafficking and this person is relate to be forced to work for produce legal service or goods it is also a case of modern slavery in supply chain. Sex trafficking may also be a case of modern slavery in supply chain if it's related to the hospitality industry"
SEX_TRAFFICKING = "Different of rape and sex assault, sex trafficking is a crime whereby the victim is trafficked or kidnapped and exploited for profit at the expense of adults or children by compelling them to perform labor or engage in commercial sex."
DRUGS_TRAFFICKING = "Commerce of illegal drugs."
CHILD_LABOR = "Child labour refers to work that children, a minor, are too young to perform or that  by its nature or circumstances can be hazardous. Unlike activities that help children develop (such as contributing to light housework or taking on a job during school holidays), child labour causes harm to a child’s health, safety or moral development."


PROMPT_TEMPLATE = f"""
**Context Definitions:**
1. Supply Chain: {SUPPLY_CHAIN_DEF}
2. Modern Slavery: {MODERN_SLAVERY_DEF} , {MODERN_SLAVERY_IN_SUPPLY_CHAIN}
3. Sex Trafficking: {SEX_TRAFFICKING}
4. Drug Trafficking: {DRUGS_TRAFFICKING}
5. Child Labor: {CHILD_LABOR}



**Task:**
Analyze the following legal case text individually based *only* on the provided text and the definitions above. Answer all questions (0-15) clearly and concisely. For questions requiring textual evidence (2, 11, 12, 13, 14, 15), extract specific quotes or paraphrased elements from the text to support 
your answer. If the information is not present in the text, state "Not mentioned in text". The final CSV must 

Provide a a short abstract of on paragraph of the central information to the case.
Abstract:
0. id: {{CASE_ID}}
1. Is this a case of modern slavery in the supply chain? (Yes / No / Not mentioned in text)
2. Extract from the text some elements that embase your question 1 answer. Give some elements from the text that embase your answer.(Provide evidence)
3. What is the service or good that is product in the case? (If none, state "None specified")
4. Who was the plaintiff? (If not mentioned, state "Not mentioned in text")
5. Who was the defendant? (If not mentioned, state "Not mentioned in text")
6. Is it related to agriculture industry? (Yes / No / Not mentioned in text)
7. Is it related to hospitality industry? (Yes / No / Not mentioned in text)
8. Is it related to technology industry? (Yes / No / Not mentioned in text)
9. Is it related to manufacturing industry? (Yes / No / Not mentioned in text)
10. Is it related to the service industry (consider only licit services)? (Yes / No / Not mentioned in text)
11. Is the case related to human trafficking? (Yes / No / Not mentioned in text). Give some elements from the text that embase your answer. (Provide evidence )
12. Is the case related to forced labor? (Yes / No / Not mentioned in text). Give some elements from the text that embase your answer. (Provide evidence )
13. Is the case related to child labor? (Yes / No / Not mentioned in text). Give some elements from the text that embase your answer. (Provide evidence )
14. Is the case related to sex trafficking? (Yes / No / Not mentioned in text). Give some elements from the text that embase your answer. (Provide evidence )
15. Is the case related to illegal substances or drugs? (Yes / No / Not mentioned in text). Give some elements from the text that embase your answer. (Provide evidence )


**Output Format:**
Provide the answers numbered 0 through 15 exactly as requested above. Start each answer on a new line prefixed with the question number (e.g., "0. id: 12345").

**Block 1 Output Formatting Instructions (Very Important):**
- Start each answer on a **new line**.
- Prefix each answer **strictly** with the question number followed by a period and a space (e.g., "0. ", "1. ", "2. ").
- For Yes/No questions (1, 6, 7, 8, 9, 10): Answer *only* with "Yes.", "No.", or "Not mentioned in text." on the line immediately after the prefix.
- For questions 11, 12, 13, 14, 15 (Yes/No + Evidence):
    - On the first line for the question (e.g., "11. "), answer *only* with "Yes.", "No.", or "Not mentioned in text."
    - If the answer is "Yes.", provide the supporting elements/evidence from the text on the *very next line*, starting that line with "Evidence: ".
    - If the answer is "No." or "Not mentioned in text.", write "Evidence: N/A" on the very next line.
- For question 2 (Evidence for Q1):
    - Directly after the "2. " prefix, provide the supporting elements/evidence from the text. If Q1 was "No." or "Not mentioned in text.", write "Evidence: N/A".
- For questions 0, 3, 4, 5: Provide the requested information directly after the prefix. If not found, state "Not mentioned in text".

**Case Text to Analyze:**
{{CASE_TEXT}}


"""

Erro: A chave de API não foi encontrada.


In [5]:
import os
from dotenv import load_dotenv

# 1. Load the .env file.
# This line reads all key-value pairs from your .env file
# and loads them as environment variables.
load_dotenv()

# 2. Read the API key from the environment variables.
# os.getenv() will now look for the API_KEY that was just loaded.
API_KEY = os.getenv('API_KEY')

# 3. Check if the key was found.
if API_KEY:
    print("Chave de API carregada com sucesso!")
    # Now you can use the API_KEY variable in your code
    # For example: make_api_call(api_key=API_KEY)
else:
    print("Erro: A chave de API não foi encontrada.")
    print("Verifique se o arquivo .env existe e se a variável 'API_KEY' está definida corretamente.")
    # It's a good practice to exit the script if a critical key is missing.
    # exit()

Erro: A chave de API não foi encontrada.
Verifique se o arquivo .env existe e se a variável 'API_KEY' está definida corretamente.


In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [9]:



def call_gemini_api(prompt, retries=3, delay=10):
    """Calls the Gemini API with retry logic."""
    # (Implementation from previous version is good, keep it here)
    for attempt in range(retries):
        try:
            response = model.generate_content(prompt, stream=False)
            if not response.candidates:
                 print(f"    [API Warning] No candidates returned. Response might be empty or blocked. Skipping.")
                 if response.prompt_feedback and response.prompt_feedback.block_reason:
                      print(f"    [API Safety Block] Reason: {response.prompt_feedback.block_reason}. Skipping this case.")
                 return None
            first_candidate = response.candidates[0]
            if first_candidate.finish_reason != 1:
                block_reason = "Unknown"
                blocked_rating = None
                for rating in first_candidate.safety_ratings:
                    if rating.probability.name in ('HIGH', 'MEDIUM'):
                         if safety_settings:
                            for setting in safety_settings:
                                if setting["category"] == rating.category.name and setting["threshold"] in ("BLOCK_MEDIUM_AND_ABOVE", "BLOCK_LOW_AND_ABOVE"):
                                     blocked_rating = rating
                                     block_reason = f"Safety Category {rating.category.name} was {rating.probability.name}"
                                     break
                         elif rating.probability.name == 'HIGH':
                            blocked_rating = rating
                            block_reason = f"Safety Category {rating.category.name} was {rating.probability.name}"
                            break
                    if blocked_rating: break
                print(f"    [API Safety Block or Finish Error] Finish Reason: {first_candidate.finish_reason.name}. Detail: {block_reason}. Skipping this case.")
                return None
            if first_candidate.content and first_candidate.content.parts:
                 try:
                     return response.text
                 except ValueError as e:
                     print(f"    [API Warning] Could not access response text although parts exist: {e}. Skipping.")
                     return None
            else:
                print(f"    [API Warning] Received empty content parts but no block/error reason. Output might be missing. Skipping.")
                return None
        except exceptions.ResourceExhausted as e:
            print(f"    [API Error] Rate limit likely hit (Attempt {attempt + 1}/{retries}): {e}. Retrying in {delay} seconds...")
            time.sleep(delay * (attempt + 1))
        except exceptions.GoogleAPIError as e:
             if hasattr(e, 'code') and e.code in [500, 503]:
                 print(f"    [API Error] Server Error ({e.code}) (Attempt {attempt + 1}/{retries}): {e}. Retrying in {delay} seconds...")
                 time.sleep(delay * (attempt + 1))
             else:
                print(f"    [API Error] Google API Error (Attempt {attempt + 1}/{retries}): {e}. Skipping this case.")
                return None
        except Exception as e:
            print(f"    [API Error] An unexpected error occurred (Attempt {attempt + 1}/{retries}): {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
    print(f"    [API Error] Failed to get response after {retries} attempts.")
    return None


# --- (*ADJUSTED* Helper Function to Parse Block 1 Answers - remains same as prev version) ---
def parse_block1(response_text):
    """
    Parses the numbered answers from Gemini's response text,
    expecting the stricter format defined in the prompt.
    Returns two dictionaries: answers and evidence.
    """
    # (Implementation from previous version is good, keep it here)
    answers = {str(i): "Error: Could not parse" for i in range(16)}
    evidence = {str(i): "" for i in [2, 11, 12, 13, 14, 15]}
    if not response_text:
        return answers, evidence
    lines = response_text.strip().split('\n')
    current_q = None
    expecting_evidence_for = None
    for i, line in enumerate(lines):
        line = line.strip()
        if not line: continue
        match = re.match(r'^(\d{1,2})\.\s+(.*)', line)
        if match:
            q_num_str = match.group(1)
            q_content = match.group(2).strip()
            expecting_evidence_for = None
            if q_num_str.isdigit() and 0 <= int(q_num_str) <= 15:
                current_q = int(q_num_str)
                answers[q_num_str] = q_content
                if 11 <= current_q <= 15:
                    answers[q_num_str] = q_content
                    if q_content.lower().startswith("yes"):
                        expecting_evidence_for = q_num_str
                elif current_q == 2:
                     evidence['2'] = q_content
                     answers['2'] = q_content
                else:
                    answers[q_num_str] = q_content
            else:
                 print(f"    [Parse Warning] Ignoring invalid question number: {q_num_str} in line: {line}")
                 current_q = None
        elif expecting_evidence_for is not None and line.startswith("Evidence:"):
             evidence_text = line.split("Evidence:", 1)[-1].strip()
             evidence[expecting_evidence_for] = evidence_text
             if answers[expecting_evidence_for].lower().startswith("yes"):
                 answers[expecting_evidence_for] += f"\n{line}"
             elif not evidence_text == "N/A":
                  print(f"    [Parse Warning] Evidence provided for Q{expecting_evidence_for} despite non-'Yes' answer: {line}")
                  answers[expecting_evidence_for] += f"\n{line} [Note: Evidence provided unexpectedly]"
             expecting_evidence_for = None
        elif current_q is not None and str(current_q) not in ['1', '2'] + [str(x) for x in range(6,16)]:
             if answers[str(current_q)] != "Error: Could not parse":
                 answers[str(current_q)] += "\n" + line
             else:
                 answers[str(current_q)] = line
    for i in range(16):
        q_num = str(i)
        if answers[q_num] == "Error: Could not parse":
            print(f"    [Parse Warning] Could not parse answer for question {q_num}")
    return answers, evidence

# --- (*NEW* Helper Function to Format Block 1 Text) ---
def format_block1_output(parsed_answers_b1, case_id):
    """Formats the parsed Block 1 answers into a single string."""
    output_lines = []
    output_lines.append(f"Analysis for Case ID: {case_id}")
    output_lines.append("-" * 20)
    for i in range(16):
        q_num_str = str(i)
        answer = parsed_answers_b1.get(q_num_str, "Error: Answer not parsed")

        # Reconstruct the prefix if it's not already there (parser might strip/alter it)
        # Basic reconstruction logic:
        prefix = f"{i}. "
        # Remove potential duplicate prefix from the answer itself before adding ours
        answer_text = re.sub(r'^(\d{1,2})\.\s+', '', answer).strip()

        # Handle Q0 specifically to include the ID properly if needed
        if i == 0:
             # If answer doesn't already contain the id explicitly.
             if not case_id in answer_text:
                 output_lines.append(f"0. id: {case_id}") # Ensure ID is shown
                 if answer_text and answer_text != "Error: Answer not parsed": # Append any other parsed text for Q0 if present
                      output_lines.append(f"   {answer_text}")
             else: # If parser included ID, use the parsed line
                  output_lines.append(f"0. {answer_text}")

        # For other questions, add the standard prefix and the answer text
        else:
             output_lines.append(f"{prefix}{answer_text}")

    return "\n".join(output_lines)


# --- (Helper Function get_yes_no remains the same) ---
def get_yes_no(text):
    """Determines Yes/No/Not Mentioned status from text, ignoring potential evidence lines."""
    # (Implementation from previous version is good, keep it here)
    first_line = text.split('\n')[0].strip().lower()
    if first_line.endswith('.'):
        first_line = first_line[:-1]
    if first_line == "yes":
        return "yes"
    elif first_line == "no":
        return "no"
    elif first_line.startswith("not mentioned"):
        return "no"
    else:
        print(f"    [CSV Logic Warning] Unexpected Yes/No input: '{text}'. Defaulting to 'no'.")
        return "no"

# --- (*MODIFIED* Helper Function to Extract Data for Block 2 CSV) ---
def extract_block2_data(parsed_answers_b1, case_id, case_name, block1_full_text): # Added block1_full_text
    """
    Extracts and formats data for the Block 2 CSV row based on parsed Block 1 answers,
    and includes the full Block 1 text.
    """
    block2 = {}

    # --- Basic Info ---
    block2['opinion'] = case_id
    block2['name'] = case_name if case_name else f"Case_{case_id}"

    # --- Modern Slavery / Supply Chain ---
    is_ht = get_yes_no(parsed_answers_b1.get('11', 'no')) == 'yes'
    is_fl = get_yes_no(parsed_answers_b1.get('12', 'no')) == 'yes'
    is_cl = get_yes_no(parsed_answers_b1.get('13', 'no')) == 'yes'
    is_st = get_yes_no(parsed_answers_b1.get('14', 'no')) == 'yes'
    #block2['modern slavery'] = "yes" if (is_ht or is_fl or is_st) else "no"
    block2['modern slavery'] = "yes" if (is_ht or is_fl or is_st or (is_cl and (is_fl or is_st or is_ht))) else "no"
    block2['modern slavery in supply chain'] = get_yes_no(parsed_answers_b1.get('1', 'no'))



    # --- Goods / Services ---
    good_or_service = parsed_answers_b1.get('3', 'None specified').split('\n')[0].lower()
    block2['service production'] = "yes" if "service" in good_or_service else "no"
    is_none = "none" in good_or_service or "not mentioned" in good_or_service
    block2['good production'] = "no" if block2['service production'] == 'yes' or is_none else "yes"

    # --- Parties ---
    block2['plaintiff names'] = parsed_answers_b1.get('4', 'Not mentioned in text')
    block2['defendant names'] = parsed_answers_b1.get('5', 'Not mentioned in text')

    # --- Industries ---
    block2['related to agriculture'] = get_yes_no(parsed_answers_b1.get('6', 'no'))
    block2['related to hospitality'] = get_yes_no(parsed_answers_b1.get('7', 'no'))
    block2['related to technology'] = get_yes_no(parsed_answers_b1.get('8', 'no'))
    block2['related to manufacturing'] = get_yes_no(parsed_answers_b1.get('9', 'no'))
    block2['related to service industry'] = get_yes_no(parsed_answers_b1.get('10', 'no'))

    # --- Specific Types ---
    block2['human trafficking'] = "yes" if is_ht else "no"
    block2['forced labor'] = "yes" if is_fl else "no"
    block2['child labor'] = "yes" if is_cl else "no"
    block2['sex trafficking'] = "yes" if is_st else "no"
    block2['drug trafficking'] = get_yes_no(parsed_answers_b1.get('15', 'no'))

    # --- *NEW*: Add Full Block 1 Text ---
    block2['block1_full_text'] = block1_full_text

    return block2



In [ ]:

# --- (*MODIFIED* Main Processing Logic) ---
print(f"Fetching CSV data from: {INPUT_CSV_URL}")
try:
    response = requests.get(INPUT_CSV_URL)
    response.raise_for_status()
    csv_data = io.StringIO(response.text)
    df = pd.read_csv(csv_data)

  
    # Criar coluna item_id sequencial
    df.reset_index(drop=True, inplace=True)
    df[id_column] = range(1, len(df) + 1)
    df = df.drop_duplicates(subset=text_column)
    print(f"Successfully loaded {len(df)} rows.")

    # Validate and prepare columns
    if id_column not in df.columns:
         raise ValueError(f"Expected ID column '{id_column}' not found in CSV.")
    if text_column not in df.columns:
         raise ValueError(f"Expected text column '{text_column}' not found in CSV.")
    if name_column not in df.columns:
        print(f"[Warning] Name column '{name_column}' not found. Using ID as name fallback.")
        df[name_column] = None
    else:
         df[name_column] = df[name_column].fillna('')

    df[text_column] = df[text_column].fillna('')
    df[id_column] = df[id_column].astype(str)

except requests.exceptions.RequestException as e:
    print(f"Error fetching data from URL: {e}")
    exit()
except pd.errors.EmptyDataError:
    print(f"Error: The CSV file from the URL seems to be empty.")
    exit()
except Exception as e:
    print(f"Error reading or processing CSV data: {e}")
    exit()



In [6]:
df = pd.read_csv("modern_slavery_NER_us_india.csv")
df = df[(df['modern_slavery'].str.lower()!='yes')==True]

In [ ]:

all_results_for_csv = []
print(f"\n--- Starting Analysis ({len(df)} cases) ---")

# --- Limit processing for testing (optional) ---
# MAX_CASES_TO_PROCESS = 3
# df = df.head(MAX_CASES_TO_PROCESS)
# print(f"[INFO] Processing only the first {len(df)} cases for testing.")
# ---

# Define the final CSV headers including the new column
fieldnames = [
    'opinion', 'name', 'modern slavery', 'modern slavery in supply chain',
    'good production', 'service production', 'plaintiff names', 'defendant names',
    'related to agriculture', 'related to hospitality', 'related to technology',
    'related to manufacturing', 'related to service industry',
    'human trafficking', 'forced labor', 'child labor', 'sex trafficking',
    'drug trafficking',
    'block1_full_text' # <-- Added new column header
]

for index, row in df.iterrows():
    case_id = str(row[id_column]) if id_column in row else f"Row_{index}_NoID"
    case_text = row.get(text_column, '')
    case_name = row.get(name_column, '') if name_column else ''

    print(f"\nProcessing Case ID: {case_id} (Row {index + 1}/{len(df)})")

    block1_formatted_text_for_output = "Skipped: Empty input text" # Default for skipped rows
    block2_row_data = {} # Initialize

    if not case_text or case_text.isspace():
        print("    Skipping row due to empty case text.")
        # Prepare placeholder for CSV
        skipped_row = {
            'opinion': case_id,
            'name': case_name if case_name else f"Case_{case_id}",
            'block1_full_text': block1_formatted_text_for_output,
            **{f: 'skipped' for f in fieldnames[2:-1]} # Fill other fields
            }
        all_results_for_csv.append(skipped_row)
        continue # Skip API call and file writing

    # Prepare the prompt
    prompt = PROMPT_TEMPLATE.replace("{CASE_TEXT}", case_text).replace("{CASE_ID}", case_id)
    prompt = prompt

    # Call the API
    response_text = call_gemini_api(prompt)

    if response_text:
        # Parse Block 1 response
        parsed_answers_b1, parsed_evidence = parse_block1(response_text)

        # Format the Block 1 output for saving/adding to CSV
        block1_formatted_text_for_output = format_block1_output(parsed_answers_b1, case_id)

        # --- Print Block 1 Output to Console ---
        print("\n--- BLOCK 1 START ---")
        print(block1_formatted_text_for_output) # Use the formatted string
        print("--- BLOCK 1 END ---")

        # --- Save Block 1 Output to TXT File ---
        txt_filename = os.path.join(BLOCK1_OUTPUT_DIR, f"block1_.txt")
        try:
            with open(txt_filename, 'a', encoding='utf-8') as f_txt:
                f_txt.write(block1_formatted_text_for_output)
            print(f"    Saved Block 1 analysis to: {txt_filename}")
        except IOError as e:
            print(f"    [Error] Failed to write Block 1 TXT file: {e}")
        except Exception as e:
             print(f"    [Error] Unexpected error writing Block 1 TXT file: {e}")


        # --- Extract data for Block 2 CSV (including the formatted Block 1 text) ---
        block2_row_data = extract_block2_data(
            parsed_answers_b1,
            case_id,
            case_name,
            block1_formatted_text_for_output # Pass the formatted text
        )
        all_results_for_csv.append(block2_row_data)

        # Optional: Short delay
        time.sleep(1.5)
    else:
        print(f"    Failed to get analysis for Case ID: {case_id}. Adding error row to CSV.")
        # Prepare placeholder for failed rows
        block1_formatted_text_for_output = "Error: API call failed or blocked"
        failed_row = {
            'opinion': case_id,
            'name': case_name if case_name else f"Case_{case_id}",
             'block1_full_text': block1_formatted_text_for_output,
             **{f: 'error' for f in fieldnames[2:-1]} # Fill other fields
             }
        all_results_for_csv.append(failed_row)


# --- Generate Block 2 CSV Output ---
if all_results_for_csv:
    print(f"\n--- Writing Block 2 CSV Output ({len(all_results_for_csv)} rows) ---")
    # Fieldnames list is already defined above including 'block1_full_text'
    try:
        with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as csvfile:
            # Use extrasaction='ignore' in case any unexpected keys somehow appear
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames, extrasaction='ignore')

            writer.writeheader()
            writer.writerows(all_results_for_csv)
        print(f"Successfully wrote analysis results to {OUTPUT_CSV_FILE}")
    except IOError as e:
        print(f"Error writing CSV file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred during CSV writing: {e}")
else:
    print("\nNo results were generated to write to CSV.")

print("\n--- Processing Complete ---")



--- Starting Analysis (1665 cases) ---

Processing Case ID: 4126582 (Row 1/1665)

--- BLOCK 1 START ---
Analysis for Case ID: 4126582
--------------------
0. id: 4126582
1. No.
2. Evidence: N/A
3. None specified
4. Edward Piotrowski
5. Secretary of Health and Human Services
6. No.
7. No.
8. No.
9. No.
10. No.
11. No.
12. No.
13. No.
14. No.
15. No.
--- BLOCK 1 END ---
    Saved Block 1 analysis to: block1_.txt

Processing Case ID: 2398649 (Row 2/1665)

--- BLOCK 1 START ---
Analysis for Case ID: 2398649
--------------------
0. id: 2398649
1. Yes.
2. Evidence:  "For at least three years before the filing of this action and continuing to the present, defendants have required plaintiffs to perform essential job duties without compensation before the start of their shifts or after their shifts should have ended."  This describes forced labor, a key element of modern slavery as defined.
3. None specified.  The case focuses on unpaid labor related to the operation of a Wisconsin penitentiar